In [489]:
import numpy as np
import numpy.matlib
from cmath import inf
np.set_printoptions(formatter={'all':lambda x: str(Fraction(x).limit_denominator())}) #设置矩阵分数运算

In [490]:
#原始模型整理
or_c = [2,1] 
or_b = [15,24]
or_a = [3,5,6,2] #系数水平输入


In [491]:
#标准型
def nomal(or_c,or_b,or_a):
    m = len(or_b)
    or_p = np.asarray(or_a).reshape(len(or_b),len(or_c)) #原系数矩阵
    B = np.matlib.eye(n=m) #初始可行基
    b = np.asarray(or_b) #限额系数
    X = np.concatenate((np.zeros(len(or_c)),b),axis=0) #初始基可行解
    A = np.concatenate((or_p,B),axis=1) #系数矩阵
    C = np.concatenate((np.asarray(or_c),np.zeros(m)),axis=0) #价值系数,初始表检验数
    return A,C,b,X



In [492]:
#检验数
def check(bA,C,Xi):
    i,o=0,0
    ck=[]
    Xi = [int(k) for k in Xi]
    c = [C[g] for g in Xi] #确定基变量系数
    c = np.asarray(c)
    A = np.delete(bA,0,axis=1)
    for a in np.nditer(A, flags = ['external_loop'], order = 'F'):
        ck.append(C[o]-np.inner(a,c))
        o += 1
    return ck,c

In [493]:
#确定换出变量和换入变量
def outin(check,bA):
    i = check.index(max(check))
    out = bA[...,i+1]
    sita = []
    for n in range(len(out)):
        if(out[n] >0):
            sita.append(bA[...,0][n]/out[n])
        elif(out[n] <= 0):
            sita.append(inf)
    sita = [float(k) for k in sita]
    o = sita.index(min(sita))
    return i,o         


In [494]:
#基变换
def iteration(i,o,bA,n):
    if(bA[o,i] != 1):
        E1 = np.matlib.eye(n)
        E1[o,...] = E1[o,...]/bA[o,i] #倍乘
        bA1 = np.dot(E1,bA)
    else:
        bA1 = bA
    for p in range(len(bA[...,i])):
        if(p == o):
            continue
        elif(bA[p,i] != 0):
            E = np.matlib.eye(n)
            E0 = np.matlib.zeros((n,n))
            E0[p,o] = bA[p,i]
            bA1 = np.dot(E-E0,bA1) #倍加
    return bA1


In [495]:
#解的判定
def judge(ck,Xi,bA):
    co = 0
    for i in range(len(ck)):
        if(i not in Xi and ck[i] == 0): 
            try:
                raise KeyboardInterrupt
            finally:
                print('有无穷解')
        if(ck[i]>0):
            if(sum(bA[...,i+1])<0):
                try:
                    raise KeyboardInterrupt
                finally:
                    print('有无界解') 
            co+=1
    return co

In [496]:
simplex = nomal(or_c,or_b,or_a)
A=simplex[0];C=simplex[1];b=simplex[2];X=simplex[3] #初始化
Xi = np.argwhere(X > 0) #初始基变量索引
bA = np.insert(A,0,b.T,axis=1) #系数矩阵与b合并
C = C.tolist()
ck=check(bA,C,Xi)[0] #检验数
dj = 0
while(True):
    co=judge(ck,Xi,bA)
    if(co>0):
        outi=outin(ck,bA) #出入变量
        bA = iteration(outi[0]+1,outi[1],bA,len(b)) #基变换
        Xi[outi[1]] = outi[0] #新基变量
        ck=check(bA,C,Xi)[0] #检验数
    else:
        break
C = check(bA,C,Xi)[1]
Xj = bA[...,0]
z = float(np.dot(C,Xj)) #目标函数值
print("目标函数值为:",z)
X = np.zeros(len(X))
for x in range(len(Xi)):
    X[int(Xi[x])] = Xj[x]
print("最优解为:",X) #最优解

目标函数值为: 8.25
最优解为: [15/4 3/4 0 0]
